Данная лабораторная работа ориентирована на знакомство с ансамблевыми методами машинного обучения с реализацией на языке программирования Python:

Реализовать решение задач классификации и регрессии пользовательских и/или модельных данных с использованием бэггинга над деревьями решений и случайного леса.
Реализовать решение задач классификации и регрессии пользовательских и/или модельных данных с использованием градиентного бустинга, алгоритмов AdaBoost, XGBoost, CatBoost и LightGBM.
Реализовать решение задач классификации и/или регрессии пользовательских и/или модельных данных с использованием стекинга.
Реализовать решение задач классификации и/или регрессии пользовательских и/или модельных данных с использованием многоуровнего стекинга.

In [56]:
from sklearn.datasets import load_breast_cancer
X, y=load_breast_cancer(return_X_y=True, as_frame=True)

In [2]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
#BaggingClassifier().get_params().keys()

dict_keys(['base_estimator', 'bootstrap', 'bootstrap_features', 'max_features', 'max_samples', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators' : [1, 2,3 ,4 ,5 ,6, 7, 8, 9, 10, 15, 20, 22, 24, 26, 31, 40, 41, 42, 43, 50, 60, 70 , 100],
    'max_samples' :[1, 2,3 ,4 ,5 ,6, 7, 8, 9, 10, 15, 20],
    'max_features' :[1, 2,3 ,4 ,5 ,6, 7, 8, 9, 10, 15, 20],
    'bootstrap' : [True, False],
    'bootstrap_features' : [True, False],
    'warm_start' :[True, False]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
tree=DecisionTreeClassifier(splitter='random', max_features='auto')
clf = BaggingClassifier(base_estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'warm_start': False, 'n_estimators': 100, 'max_samples': 20, 'max_features': 8, 'bootstrap_features': True, 'bootstrap': False}
Лучшее качество модели: 0.945054945054945


In [24]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.9385964912280702
precision =  0.9722222222222222


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators':[10, 15, 20, 22, 24, 26, 31, 40, 41, 42, 43, 50, 60, 70 , 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40, 45,  50, 55, 60],
    'min_samples_split': [ 2, 3, 4,  5, 10, 15, 20, 25, 30 , 40],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7,8, 10, 15, 20, 25, 30, 35, 40],
    'max_features': ['auto', 'sqrt', 'log2'],
    'warm_start' :[True, False],
    'bootstrap' : [True, False]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
dtree = RandomForestClassifier()
random_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'warm_start': False, 'n_estimators': 43, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'log2', 'max_depth': None, 'criterion': 'entropy', 'bootstrap': False}
Лучшее качество модели: 0.9670329670329672


In [28]:
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.9649122807017544
precision =  0.9861111111111112


In [33]:
from sklearn.ensemble import AdaBoostClassifier
param_dist = {
    'n_estimators':[10, 15, 20, 22, 24, 26, 31, 40, 41, 42, 43, 50, 60, 70 , 100],
    'algorithm' :['SAMME', 'SAMME.R'],
    'learning_rate' :[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.3, 1.7, 2.0, 3, 4, 5, 6, 7, 10, 20, 30]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
tree=DecisionTreeClassifier()
clf = AdaBoostClassifier(base_estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'n_estimators': 10, 'learning_rate': 0.7, 'algorithm': 'SAMME.R'}
Лучшее качество модели: 0.9296703296703297


In [34]:
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.9035087719298246
precision =  0.8888888888888888


In [38]:

from sklearn.ensemble import AdaBoostClassifier
param_dist = {
    'n_estimators' : [10, 15, 20, 22, 24, 26, 31, 40, 41, 42, 43, 50, 60, 70 , 100],
    'algorithm' : ['SAMME', 'SAMME.R'],
    'learning_rate' : [0.6, 0.7, 0.8, 0.9, 1.0, 1.3, 1.7]
}
tree=DecisionTreeClassifier(max_depth=2)
clf = AdaBoostClassifier(base_estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'n_estimators': 70, 'learning_rate': 1.0, 'algorithm': 'SAMME'}
Лучшее качество модели: 0.9758241758241759


In [39]:
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.9736842105263158
precision =  1.0


In [42]:
from sklearn.ensemble import AdaBoostClassifier
param_dist = {
    'n_estimators' : [40, 41, 42, 43, 50, 60, 70 , 100],
    'algorithm' : ['SAMME', 'SAMME.R'],
    'learning_rate' : [0.6, 0.7, 0.8, 0.9, 1.0, 1.3, 1.7]
}
tree=DecisionTreeClassifier(max_depth=3)
clf = AdaBoostClassifier(base_estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

C:\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:656: RuntimeWarning: divide by zero encountered in log
  np.log(sample_weight)
C:\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:656: RuntimeWarning: divide by zero encountered in log
  np.log(sample_weight)
C:\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:656: RuntimeWarning: divide by zero encountered in log
  np.log(sample_weight)
C:\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:656: RuntimeWarning: divide by zero encountered in log
  np.log(sample_weight)
C:\anaconda3\lib\site-packages\sklearn\ensemble\_weight_boosting.py:656: RuntimeWarning: divide by zero encountered in log
  np.log(sample_weight)


Лучшие параметры: {'n_estimators': 50, 'learning_rate': 1.3, 'algorithm': 'SAMME'}
Лучшее качество модели: 0.9780219780219781


In [43]:
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.956140350877193
precision =  0.9722222222222222


In [44]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [58]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, max_depth=3, learning_rate=1, objective='binary:logistic')
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

accuracy =  0.956140350877193
precision =  0.9861111111111112


In [50]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [51]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 285, number of negative: 170
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4542
[LightGBM] [Info] Number of data points in the train set: 455, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.626374 -> initscore=0.516691
[LightGBM] [Info] Start training from score 0.516691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [52]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('accuracy = ', accuracy_score(y_pred, y_test))
print('precision = ', precision_score(y_pred, y_test))

Learning rate set to 0.00736
0:	learn: 0.6806080	total: 160ms	remaining: 2m 39s
1:	learn: 0.6690202	total: 165ms	remaining: 1m 22s
2:	learn: 0.6578043	total: 169ms	remaining: 56.3s
3:	learn: 0.6465633	total: 175ms	remaining: 43.5s
4:	learn: 0.6345548	total: 179ms	remaining: 35.7s
5:	learn: 0.6253510	total: 184ms	remaining: 30.5s
6:	learn: 0.6138483	total: 189ms	remaining: 26.9s
7:	learn: 0.6029704	total: 194ms	remaining: 24.1s
8:	learn: 0.5921247	total: 199ms	remaining: 21.9s
9:	learn: 0.5823277	total: 204ms	remaining: 20.2s
10:	learn: 0.5723340	total: 209ms	remaining: 18.8s
11:	learn: 0.5628507	total: 213ms	remaining: 17.6s
12:	learn: 0.5546941	total: 218ms	remaining: 16.6s
13:	learn: 0.5462362	total: 223ms	remaining: 15.7s
14:	learn: 0.5374813	total: 228ms	remaining: 15s
15:	learn: 0.5284222	total: 233ms	remaining: 14.3s
16:	learn: 0.5205587	total: 238ms	remaining: 13.7s
17:	learn: 0.5123785	total: 242ms	remaining: 13.2s
18:	learn: 0.5040587	total: 247ms	remaining: 12.8s
19:	learn: 0

176:	learn: 0.0946638	total: 1.05s	remaining: 4.88s
177:	learn: 0.0939313	total: 1.05s	remaining: 4.87s
178:	learn: 0.0933971	total: 1.06s	remaining: 4.86s
179:	learn: 0.0926860	total: 1.07s	remaining: 4.86s
180:	learn: 0.0922370	total: 1.07s	remaining: 4.84s
181:	learn: 0.0918659	total: 1.07s	remaining: 4.83s
182:	learn: 0.0911985	total: 1.08s	remaining: 4.83s
183:	learn: 0.0906912	total: 1.08s	remaining: 4.81s
184:	learn: 0.0899366	total: 1.09s	remaining: 4.8s
185:	learn: 0.0895976	total: 1.09s	remaining: 4.79s
186:	learn: 0.0889175	total: 1.1s	remaining: 4.78s
187:	learn: 0.0884851	total: 1.1s	remaining: 4.77s
188:	learn: 0.0879296	total: 1.11s	remaining: 4.76s
189:	learn: 0.0872452	total: 1.11s	remaining: 4.75s
190:	learn: 0.0866837	total: 1.12s	remaining: 4.74s
191:	learn: 0.0861988	total: 1.12s	remaining: 4.73s
192:	learn: 0.0857253	total: 1.13s	remaining: 4.72s
193:	learn: 0.0851695	total: 1.13s	remaining: 4.71s
194:	learn: 0.0847459	total: 1.14s	remaining: 4.71s
195:	learn: 0.0

339:	learn: 0.0432670	total: 1.94s	remaining: 3.76s
340:	learn: 0.0431386	total: 1.94s	remaining: 3.75s
341:	learn: 0.0430035	total: 1.95s	remaining: 3.75s
342:	learn: 0.0428659	total: 1.95s	remaining: 3.74s
343:	learn: 0.0426859	total: 1.96s	remaining: 3.73s
344:	learn: 0.0424815	total: 1.96s	remaining: 3.73s
345:	learn: 0.0423494	total: 1.97s	remaining: 3.72s
346:	learn: 0.0421634	total: 1.97s	remaining: 3.71s
347:	learn: 0.0420144	total: 1.98s	remaining: 3.71s
348:	learn: 0.0418578	total: 1.99s	remaining: 3.7s
349:	learn: 0.0417805	total: 1.99s	remaining: 3.7s
350:	learn: 0.0416040	total: 2s	remaining: 3.69s
351:	learn: 0.0414448	total: 2s	remaining: 3.68s
352:	learn: 0.0412940	total: 2s	remaining: 3.67s
353:	learn: 0.0411391	total: 2.01s	remaining: 3.67s
354:	learn: 0.0410405	total: 2.02s	remaining: 3.66s
355:	learn: 0.0408972	total: 2.02s	remaining: 3.65s
356:	learn: 0.0407543	total: 2.02s	remaining: 3.65s
357:	learn: 0.0406020	total: 2.03s	remaining: 3.64s
358:	learn: 0.0404348	t

500:	learn: 0.0259660	total: 2.82s	remaining: 2.81s
501:	learn: 0.0259375	total: 2.83s	remaining: 2.81s
502:	learn: 0.0258609	total: 2.84s	remaining: 2.8s
503:	learn: 0.0258087	total: 2.84s	remaining: 2.8s
504:	learn: 0.0257542	total: 2.85s	remaining: 2.79s
505:	learn: 0.0257018	total: 2.85s	remaining: 2.78s
506:	learn: 0.0256497	total: 2.86s	remaining: 2.78s
507:	learn: 0.0255368	total: 2.86s	remaining: 2.77s
508:	learn: 0.0254747	total: 2.87s	remaining: 2.77s
509:	learn: 0.0253900	total: 2.87s	remaining: 2.76s
510:	learn: 0.0253439	total: 2.88s	remaining: 2.75s
511:	learn: 0.0252631	total: 2.88s	remaining: 2.75s
512:	learn: 0.0251727	total: 2.89s	remaining: 2.74s
513:	learn: 0.0251024	total: 2.89s	remaining: 2.74s
514:	learn: 0.0250622	total: 2.9s	remaining: 2.73s
515:	learn: 0.0249712	total: 2.9s	remaining: 2.72s
516:	learn: 0.0248888	total: 2.91s	remaining: 2.72s
517:	learn: 0.0248002	total: 2.92s	remaining: 2.71s
518:	learn: 0.0247227	total: 2.92s	remaining: 2.71s
519:	learn: 0.02

672:	learn: 0.0169030	total: 3.71s	remaining: 1.8s
673:	learn: 0.0168517	total: 3.72s	remaining: 1.8s
674:	learn: 0.0168135	total: 3.72s	remaining: 1.79s
675:	learn: 0.0167762	total: 3.73s	remaining: 1.79s
676:	learn: 0.0167282	total: 3.73s	remaining: 1.78s
677:	learn: 0.0167115	total: 3.74s	remaining: 1.78s
678:	learn: 0.0166848	total: 3.75s	remaining: 1.77s
679:	learn: 0.0166572	total: 3.75s	remaining: 1.76s
680:	learn: 0.0166083	total: 3.75s	remaining: 1.76s
681:	learn: 0.0165720	total: 3.76s	remaining: 1.75s
682:	learn: 0.0165386	total: 3.77s	remaining: 1.75s
683:	learn: 0.0164929	total: 3.77s	remaining: 1.74s
684:	learn: 0.0164408	total: 3.78s	remaining: 1.74s
685:	learn: 0.0164075	total: 3.78s	remaining: 1.73s
686:	learn: 0.0163801	total: 3.79s	remaining: 1.73s
687:	learn: 0.0163281	total: 3.79s	remaining: 1.72s
688:	learn: 0.0162813	total: 3.8s	remaining: 1.71s
689:	learn: 0.0162473	total: 3.8s	remaining: 1.71s
690:	learn: 0.0162238	total: 3.81s	remaining: 1.7s
691:	learn: 0.016

860:	learn: 0.0116130	total: 4.79s	remaining: 773ms
861:	learn: 0.0115868	total: 4.8s	remaining: 768ms
862:	learn: 0.0115648	total: 4.8s	remaining: 763ms
863:	learn: 0.0115520	total: 4.81s	remaining: 757ms
864:	learn: 0.0115325	total: 4.81s	remaining: 751ms
865:	learn: 0.0114954	total: 4.82s	remaining: 746ms
866:	learn: 0.0114544	total: 4.83s	remaining: 740ms
867:	learn: 0.0114282	total: 4.83s	remaining: 735ms
868:	learn: 0.0114038	total: 4.83s	remaining: 729ms
869:	learn: 0.0113957	total: 4.84s	remaining: 723ms
870:	learn: 0.0113802	total: 4.84s	remaining: 718ms
871:	learn: 0.0113593	total: 4.85s	remaining: 712ms
872:	learn: 0.0113532	total: 4.86s	remaining: 706ms
873:	learn: 0.0113395	total: 4.86s	remaining: 701ms
874:	learn: 0.0113150	total: 4.87s	remaining: 695ms
875:	learn: 0.0112950	total: 4.87s	remaining: 689ms
876:	learn: 0.0112749	total: 4.88s	remaining: 684ms
877:	learn: 0.0112585	total: 4.88s	remaining: 678ms
878:	learn: 0.0112400	total: 4.88s	remaining: 673ms
879:	learn: 0.

In [1]:
from sklearn.datasets import fetch_california_housing
X, y=fetch_california_housing(return_X_y=True, as_frame=True)
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators' : [1, 2,3 ,4 ,5 ,6, 7, 8, 9, 10, 15, 20, 22, 24, 26, 31, 40, 41, 42, 43, 50, 60, 70 , 100],
    'max_samples' :[1, 2,3 ,4 ,5 ,6, 7, 8, 9, 10, 15, 20],
    'max_features' :[1, 2,3 ,4 ,5 ,6, 7, 8],
    'bootstrap' : [True, False],
    'bootstrap_features' : [True, False],
    'warm_start' :[True, False]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
tree=DecisionTreeRegressor(splitter='random', max_features='sqrt')
clf = BaggingRegressor(estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=100, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'warm_start': False, 'n_estimators': 100, 'max_samples': 20, 'max_features': 8, 'bootstrap_features': False, 'bootstrap': False}
Лучшее качество модели: -0.7613231552635674


In [16]:
from sklearn.metrics import mean_squared_error
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.7677074003220391


In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators':[50, 60, 70],
    'criterion': ['squared_error'],
    'max_depth': [40, 45,  50, 55, 60],
    'min_samples_split': [10, 15, 20, 25],
    'min_samples_leaf': [ 10, 15, 20, 25],
    'max_features': ['sqrt'],
    'warm_start' :[True, False],
    'bootstrap' : [True]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
dtree = RandomForestRegressor()
random_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'warm_start': True, 'n_estimators': 70, 'min_samples_split': 15, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': 60, 'criterion': 'squared_error', 'bootstrap': True}
Лучшее качество модели: -0.28248589305441846


In [23]:
from sklearn.metrics import mean_squared_error
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.27827812071327945


C:\Users\Алекхаил\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py:435: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn(


In [26]:
from sklearn.ensemble import AdaBoostRegressor
param_dist = {
    'n_estimators':[40, 41, 42, 43, 50, 60, 70 , 100],
    'learning_rate' :[ 0.8, 0.9, 1.0, 1.3, 1.7, 2.0]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
tree=DecisionTreeRegressor()
clf = AdaBoostRegressor(estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'n_estimators': 60, 'learning_rate': 1.7}
Лучшее качество модели: -0.23628560990448472


In [27]:
from sklearn.metrics import mean_squared_error
model=random_search.best_estimator_
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.23232264499791663


In [28]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=1, objective='reg:squarederror')
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.24958666054540926


In [29]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))


Learning rate set to 0.063766
0:	learn: 1.1147097	total: 192ms	remaining: 3m 11s
1:	learn: 1.0778029	total: 197ms	remaining: 1m 38s
2:	learn: 1.0430071	total: 202ms	remaining: 1m 6s
3:	learn: 1.0106614	total: 207ms	remaining: 51.5s
4:	learn: 0.9819674	total: 213ms	remaining: 42.3s
5:	learn: 0.9546991	total: 217ms	remaining: 36s
6:	learn: 0.9281868	total: 223ms	remaining: 31.6s
7:	learn: 0.9037831	total: 228ms	remaining: 28.2s
8:	learn: 0.8825569	total: 232ms	remaining: 25.5s
9:	learn: 0.8608727	total: 237ms	remaining: 23.5s
10:	learn: 0.8418439	total: 242ms	remaining: 21.8s
11:	learn: 0.8242003	total: 247ms	remaining: 20.3s
12:	learn: 0.8085311	total: 252ms	remaining: 19.1s
13:	learn: 0.7918148	total: 256ms	remaining: 18s
14:	learn: 0.7785217	total: 261ms	remaining: 17.1s
15:	learn: 0.7662624	total: 265ms	remaining: 16.3s
16:	learn: 0.7528313	total: 270ms	remaining: 15.6s
17:	learn: 0.7416752	total: 275ms	remaining: 15s
18:	learn: 0.7316411	total: 279ms	remaining: 14.4s
19:	learn: 0.72

163:	learn: 0.4840241	total: 890ms	remaining: 4.53s
164:	learn: 0.4835015	total: 897ms	remaining: 4.54s
165:	learn: 0.4826047	total: 902ms	remaining: 4.53s
166:	learn: 0.4823244	total: 907ms	remaining: 4.52s
167:	learn: 0.4818147	total: 911ms	remaining: 4.51s
168:	learn: 0.4812123	total: 916ms	remaining: 4.5s
169:	learn: 0.4807361	total: 923ms	remaining: 4.5s
170:	learn: 0.4802340	total: 928ms	remaining: 4.5s
171:	learn: 0.4797409	total: 933ms	remaining: 4.49s
172:	learn: 0.4791457	total: 938ms	remaining: 4.48s
173:	learn: 0.4788243	total: 943ms	remaining: 4.48s
174:	learn: 0.4781719	total: 948ms	remaining: 4.47s
175:	learn: 0.4777012	total: 952ms	remaining: 4.46s
176:	learn: 0.4773676	total: 957ms	remaining: 4.45s
177:	learn: 0.4769651	total: 963ms	remaining: 4.45s
178:	learn: 0.4765327	total: 966ms	remaining: 4.43s
179:	learn: 0.4759817	total: 971ms	remaining: 4.42s
180:	learn: 0.4755942	total: 976ms	remaining: 4.41s
181:	learn: 0.4753260	total: 981ms	remaining: 4.41s
182:	learn: 0.4

360:	learn: 0.4223274	total: 1.75s	remaining: 3.11s
361:	learn: 0.4221836	total: 1.76s	remaining: 3.1s
362:	learn: 0.4219791	total: 1.76s	remaining: 3.09s
363:	learn: 0.4218025	total: 1.77s	remaining: 3.09s
364:	learn: 0.4216847	total: 1.77s	remaining: 3.08s
365:	learn: 0.4214848	total: 1.77s	remaining: 3.07s
366:	learn: 0.4213220	total: 1.78s	remaining: 3.07s
367:	learn: 0.4210215	total: 1.78s	remaining: 3.06s
368:	learn: 0.4208535	total: 1.79s	remaining: 3.05s
369:	learn: 0.4207169	total: 1.79s	remaining: 3.05s
370:	learn: 0.4205425	total: 1.79s	remaining: 3.04s
371:	learn: 0.4202821	total: 1.8s	remaining: 3.03s
372:	learn: 0.4200669	total: 1.8s	remaining: 3.03s
373:	learn: 0.4198612	total: 1.8s	remaining: 3.02s
374:	learn: 0.4197652	total: 1.81s	remaining: 3.01s
375:	learn: 0.4194710	total: 1.81s	remaining: 3s
376:	learn: 0.4192714	total: 1.81s	remaining: 3s
377:	learn: 0.4189897	total: 1.82s	remaining: 2.99s
378:	learn: 0.4188333	total: 1.82s	remaining: 2.98s
379:	learn: 0.4185757	

523:	learn: 0.3946196	total: 2.45s	remaining: 2.22s
524:	learn: 0.3944751	total: 2.45s	remaining: 2.22s
525:	learn: 0.3943095	total: 2.45s	remaining: 2.21s
526:	learn: 0.3941582	total: 2.46s	remaining: 2.21s
527:	learn: 0.3939963	total: 2.46s	remaining: 2.2s
528:	learn: 0.3938229	total: 2.47s	remaining: 2.2s
529:	learn: 0.3936528	total: 2.47s	remaining: 2.19s
530:	learn: 0.3935167	total: 2.47s	remaining: 2.18s
531:	learn: 0.3933753	total: 2.48s	remaining: 2.18s
532:	learn: 0.3932005	total: 2.48s	remaining: 2.17s
533:	learn: 0.3930429	total: 2.48s	remaining: 2.17s
534:	learn: 0.3928677	total: 2.49s	remaining: 2.16s
535:	learn: 0.3927327	total: 2.49s	remaining: 2.16s
536:	learn: 0.3925838	total: 2.5s	remaining: 2.15s
537:	learn: 0.3924145	total: 2.5s	remaining: 2.15s
538:	learn: 0.3923050	total: 2.5s	remaining: 2.14s
539:	learn: 0.3920838	total: 2.51s	remaining: 2.13s
540:	learn: 0.3919081	total: 2.51s	remaining: 2.13s
541:	learn: 0.3917715	total: 2.51s	remaining: 2.12s
542:	learn: 0.391

691:	learn: 0.3731155	total: 3.14s	remaining: 1.4s
692:	learn: 0.3730157	total: 3.15s	remaining: 1.39s
693:	learn: 0.3728969	total: 3.15s	remaining: 1.39s
694:	learn: 0.3727560	total: 3.16s	remaining: 1.39s
695:	learn: 0.3726732	total: 3.16s	remaining: 1.38s
696:	learn: 0.3725638	total: 3.16s	remaining: 1.38s
697:	learn: 0.3724351	total: 3.17s	remaining: 1.37s
698:	learn: 0.3723370	total: 3.17s	remaining: 1.36s
699:	learn: 0.3721924	total: 3.17s	remaining: 1.36s
700:	learn: 0.3720842	total: 3.18s	remaining: 1.35s
701:	learn: 0.3719783	total: 3.18s	remaining: 1.35s
702:	learn: 0.3718860	total: 3.19s	remaining: 1.35s
703:	learn: 0.3718075	total: 3.19s	remaining: 1.34s
704:	learn: 0.3717180	total: 3.19s	remaining: 1.34s
705:	learn: 0.3715833	total: 3.2s	remaining: 1.33s
706:	learn: 0.3714794	total: 3.2s	remaining: 1.33s
707:	learn: 0.3713437	total: 3.21s	remaining: 1.32s
708:	learn: 0.3712750	total: 3.21s	remaining: 1.32s
709:	learn: 0.3711084	total: 3.21s	remaining: 1.31s
710:	learn: 0.3

878:	learn: 0.3537509	total: 3.83s	remaining: 528ms
879:	learn: 0.3535798	total: 3.84s	remaining: 524ms
880:	learn: 0.3534721	total: 3.84s	remaining: 519ms
881:	learn: 0.3534113	total: 3.85s	remaining: 515ms
882:	learn: 0.3533479	total: 3.85s	remaining: 510ms
883:	learn: 0.3532909	total: 3.85s	remaining: 506ms
884:	learn: 0.3531941	total: 3.86s	remaining: 501ms
885:	learn: 0.3529918	total: 3.86s	remaining: 497ms
886:	learn: 0.3529096	total: 3.87s	remaining: 492ms
887:	learn: 0.3528553	total: 3.87s	remaining: 488ms
888:	learn: 0.3526707	total: 3.87s	remaining: 484ms
889:	learn: 0.3526166	total: 3.88s	remaining: 479ms
890:	learn: 0.3525310	total: 3.88s	remaining: 475ms
891:	learn: 0.3523812	total: 3.88s	remaining: 470ms
892:	learn: 0.3522824	total: 3.89s	remaining: 466ms
893:	learn: 0.3522225	total: 3.89s	remaining: 461ms
894:	learn: 0.3521464	total: 3.89s	remaining: 457ms
895:	learn: 0.3520723	total: 3.9s	remaining: 452ms
896:	learn: 0.3519817	total: 3.9s	remaining: 448ms
897:	learn: 0.

In [30]:
from lightgbm import LGBMRegressor
model = LGBMRegressor()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000311 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.071947
MSE =  0.21484820209255617


In [3]:
from xgboost import XGBRegressor
modelXGB = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=1, objective='reg:squarederror')
modelXGB.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [4]:
from lightgbm import LGBMRegressor
modelLGBM = LGBMRegressor()
modelLGBM.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000415 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1838
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 8
[LightGBM] [Info] Start training from score 2.071947


LGBMRegressor()

In [8]:
import pandas as pd
X_train_stacking=pd.DataFrame()
X_train_stacking[0]=pd.DataFrame(modelLGBM.predict(X_train))
X_train_stacking[1]=pd.DataFrame(modelXGB.predict(X_train))
X_train_stacking.head()

,0,1
0,1.369736,1.314722
1,3.581204,3.320556
2,1.920001,1.895939
3,0.970146,0.855651
4,1.321432,1.222258


In [9]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(X_train_stacking, y_train)

Learning rate set to 0.063766
0:	learn: 1.0932396	total: 172ms	remaining: 2m 51s
1:	learn: 1.0351063	total: 177ms	remaining: 1m 28s
2:	learn: 0.9811091	total: 181ms	remaining: 1m
3:	learn: 0.9310521	total: 185ms	remaining: 46s
4:	learn: 0.8844005	total: 188ms	remaining: 37.4s
5:	learn: 0.8412048	total: 192ms	remaining: 31.8s
6:	learn: 0.8010299	total: 196ms	remaining: 27.8s
7:	learn: 0.7636005	total: 201ms	remaining: 25s
8:	learn: 0.7291217	total: 205ms	remaining: 22.5s
9:	learn: 0.6970485	total: 209ms	remaining: 20.7s
10:	learn: 0.6676386	total: 212ms	remaining: 19.1s
11:	learn: 0.6404952	total: 216ms	remaining: 17.8s
12:	learn: 0.6155721	total: 219ms	remaining: 16.7s
13:	learn: 0.5925210	total: 223ms	remaining: 15.7s
14:	learn: 0.5719329	total: 228ms	remaining: 15s
15:	learn: 0.5530498	total: 231ms	remaining: 14.2s
16:	learn: 0.5356363	total: 235ms	remaining: 13.6s
17:	learn: 0.5198275	total: 239ms	remaining: 13s
18:	learn: 0.5056447	total: 242ms	remaining: 12.5s
19:	learn: 0.4923138

207:	learn: 0.3700994	total: 886ms	remaining: 3.37s
208:	learn: 0.3700695	total: 889ms	remaining: 3.37s
209:	learn: 0.3700228	total: 894ms	remaining: 3.36s
210:	learn: 0.3699850	total: 898ms	remaining: 3.36s
211:	learn: 0.3699615	total: 902ms	remaining: 3.35s
212:	learn: 0.3699344	total: 906ms	remaining: 3.35s
213:	learn: 0.3699109	total: 912ms	remaining: 3.35s
214:	learn: 0.3698509	total: 917ms	remaining: 3.35s
215:	learn: 0.3698300	total: 922ms	remaining: 3.35s
216:	learn: 0.3697584	total: 925ms	remaining: 3.34s
217:	learn: 0.3697392	total: 929ms	remaining: 3.33s
218:	learn: 0.3697260	total: 934ms	remaining: 3.33s
219:	learn: 0.3697047	total: 937ms	remaining: 3.32s
220:	learn: 0.3696866	total: 941ms	remaining: 3.32s
221:	learn: 0.3696502	total: 944ms	remaining: 3.31s
222:	learn: 0.3696303	total: 948ms	remaining: 3.3s
223:	learn: 0.3696178	total: 951ms	remaining: 3.29s
224:	learn: 0.3695920	total: 955ms	remaining: 3.29s
225:	learn: 0.3695437	total: 958ms	remaining: 3.28s
226:	learn: 0

417:	learn: 0.3633942	total: 1.59s	remaining: 2.22s
418:	learn: 0.3633830	total: 1.6s	remaining: 2.21s
419:	learn: 0.3633710	total: 1.6s	remaining: 2.21s
420:	learn: 0.3633315	total: 1.6s	remaining: 2.21s
421:	learn: 0.3632920	total: 1.61s	remaining: 2.2s
422:	learn: 0.3632608	total: 1.61s	remaining: 2.2s
423:	learn: 0.3632319	total: 1.62s	remaining: 2.19s
424:	learn: 0.3632175	total: 1.62s	remaining: 2.19s
425:	learn: 0.3631803	total: 1.62s	remaining: 2.19s
426:	learn: 0.3631711	total: 1.63s	remaining: 2.18s
427:	learn: 0.3631539	total: 1.63s	remaining: 2.18s
428:	learn: 0.3631166	total: 1.63s	remaining: 2.17s
429:	learn: 0.3630759	total: 1.64s	remaining: 2.17s
430:	learn: 0.3630559	total: 1.64s	remaining: 2.17s
431:	learn: 0.3630412	total: 1.64s	remaining: 2.16s
432:	learn: 0.3630233	total: 1.65s	remaining: 2.16s
433:	learn: 0.3629958	total: 1.65s	remaining: 2.15s
434:	learn: 0.3629826	total: 1.65s	remaining: 2.15s
435:	learn: 0.3629717	total: 1.66s	remaining: 2.14s
436:	learn: 0.362

618:	learn: 0.3590648	total: 2.28s	remaining: 1.41s
619:	learn: 0.3590405	total: 2.29s	remaining: 1.4s
620:	learn: 0.3590237	total: 2.29s	remaining: 1.4s
621:	learn: 0.3589905	total: 2.29s	remaining: 1.39s
622:	learn: 0.3589833	total: 2.3s	remaining: 1.39s
623:	learn: 0.3589706	total: 2.3s	remaining: 1.39s
624:	learn: 0.3589510	total: 2.31s	remaining: 1.38s
625:	learn: 0.3589176	total: 2.31s	remaining: 1.38s
626:	learn: 0.3589010	total: 2.31s	remaining: 1.38s
627:	learn: 0.3588883	total: 2.31s	remaining: 1.37s
628:	learn: 0.3588757	total: 2.32s	remaining: 1.37s
629:	learn: 0.3588498	total: 2.32s	remaining: 1.36s
630:	learn: 0.3588218	total: 2.33s	remaining: 1.36s
631:	learn: 0.3588006	total: 2.33s	remaining: 1.36s
632:	learn: 0.3587938	total: 2.33s	remaining: 1.35s
633:	learn: 0.3587803	total: 2.34s	remaining: 1.35s
634:	learn: 0.3587552	total: 2.34s	remaining: 1.34s
635:	learn: 0.3587182	total: 2.34s	remaining: 1.34s
636:	learn: 0.3586958	total: 2.35s	remaining: 1.34s
637:	learn: 0.35

809:	learn: 0.3558268	total: 2.98s	remaining: 698ms
810:	learn: 0.3558228	total: 2.98s	remaining: 694ms
811:	learn: 0.3558112	total: 2.98s	remaining: 691ms
812:	learn: 0.3557932	total: 2.99s	remaining: 687ms
813:	learn: 0.3557795	total: 2.99s	remaining: 683ms
814:	learn: 0.3557746	total: 2.99s	remaining: 680ms
815:	learn: 0.3557714	total: 3s	remaining: 676ms
816:	learn: 0.3557444	total: 3s	remaining: 673ms
817:	learn: 0.3557418	total: 3.01s	remaining: 669ms
818:	learn: 0.3557084	total: 3.01s	remaining: 665ms
819:	learn: 0.3557034	total: 3.01s	remaining: 662ms
820:	learn: 0.3556938	total: 3.02s	remaining: 658ms
821:	learn: 0.3556785	total: 3.02s	remaining: 654ms
822:	learn: 0.3556687	total: 3.02s	remaining: 651ms
823:	learn: 0.3556445	total: 3.03s	remaining: 647ms
824:	learn: 0.3556243	total: 3.03s	remaining: 644ms
825:	learn: 0.3556195	total: 3.04s	remaining: 640ms
826:	learn: 0.3556133	total: 3.04s	remaining: 636ms
827:	learn: 0.3556093	total: 3.04s	remaining: 632ms
828:	learn: 0.3556

In [10]:
def stacking_model_predict(X) :
    features=pd.DataFrame()
    features[0]=pd.DataFrame(modelLGBM.predict(X))
    features[1]=pd.DataFrame(modelXGB.predict(X))
    return model.predict(features)

In [12]:
from sklearn.metrics import mean_squared_error
y_pred=stacking_model_predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.22558503353082304


In [15]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'n_estimators':[40, 41, 42, 43, 50, 60, 70 , 100],
    'learning_rate' :[ 0.8, 0.9, 1.0, 1.3, 1.7, 2.0]
    #'min_impurity_decrease' : uniform(0, 0.2),
    #'ccp_alpha' : uniform(0, 0.2)
}
tree=DecisionTreeRegressor(max_depth=2)
clf = AdaBoostRegressor(estimator=tree)
random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)
print("Лучшие параметры:", random_search.best_params_)
print("Лучшее качество модели:", random_search.best_score_)

Лучшие параметры: {'n_estimators': 50, 'learning_rate': 0.8}
Лучшее качество модели: -0.7349734652201627


In [17]:
model_for_stacking=random_search.best_estimator_

In [18]:
new_X_train=pd.DataFrame()
new_X_train[0]=pd.DataFrame(model.predict(X_train_stacking))
new_X_train[1]=pd.DataFrame(model_for_stacking.predict(X_train))
new_X_train.head()

,0,1
0,1.281294,2.069987
1,3.493696,3.097270
2,1.910020,2.544984
3,0.877553,1.599476
4,1.234962,2.544984


In [19]:
meta_regressor=CatBoostRegressor()
meta_regressor.fit(new_X_train, y_train)

Learning rate set to 0.063766
0:	learn: 1.0923791	total: 3.52ms	remaining: 3.52s
1:	learn: 1.0333423	total: 6.92ms	remaining: 3.46s
2:	learn: 0.9780445	total: 9.98ms	remaining: 3.32s
3:	learn: 0.9270895	total: 13.2ms	remaining: 3.28s
4:	learn: 0.8791679	total: 16.4ms	remaining: 3.26s
5:	learn: 0.8370300	total: 19.5ms	remaining: 3.22s
6:	learn: 0.7955630	total: 22.6ms	remaining: 3.21s
7:	learn: 0.7572158	total: 25.7ms	remaining: 3.19s
8:	learn: 0.7214008	total: 28.9ms	remaining: 3.18s
9:	learn: 0.6891887	total: 32ms	remaining: 3.17s
10:	learn: 0.6591686	total: 35.2ms	remaining: 3.17s
11:	learn: 0.6318760	total: 38.3ms	remaining: 3.16s
12:	learn: 0.6057105	total: 41.5ms	remaining: 3.15s
13:	learn: 0.5821852	total: 44.7ms	remaining: 3.15s
14:	learn: 0.5606044	total: 47.9ms	remaining: 3.15s
15:	learn: 0.5404968	total: 51.2ms	remaining: 3.15s
16:	learn: 0.5220293	total: 54.3ms	remaining: 3.14s
17:	learn: 0.5050833	total: 57.3ms	remaining: 3.12s
18:	learn: 0.4903065	total: 60.5ms	remaining: 

202:	learn: 0.3431612	total: 662ms	remaining: 2.6s
203:	learn: 0.3431501	total: 665ms	remaining: 2.6s
204:	learn: 0.3431120	total: 669ms	remaining: 2.59s
205:	learn: 0.3430803	total: 672ms	remaining: 2.59s
206:	learn: 0.3430576	total: 675ms	remaining: 2.59s
207:	learn: 0.3430263	total: 679ms	remaining: 2.58s
208:	learn: 0.3430025	total: 683ms	remaining: 2.58s
209:	learn: 0.3429840	total: 687ms	remaining: 2.58s
210:	learn: 0.3429336	total: 690ms	remaining: 2.58s
211:	learn: 0.3429088	total: 695ms	remaining: 2.58s
212:	learn: 0.3428682	total: 698ms	remaining: 2.58s
213:	learn: 0.3428367	total: 701ms	remaining: 2.57s
214:	learn: 0.3428137	total: 704ms	remaining: 2.57s
215:	learn: 0.3427886	total: 707ms	remaining: 2.57s
216:	learn: 0.3427472	total: 710ms	remaining: 2.56s
217:	learn: 0.3426950	total: 714ms	remaining: 2.56s
218:	learn: 0.3426732	total: 717ms	remaining: 2.56s
219:	learn: 0.3426536	total: 720ms	remaining: 2.55s
220:	learn: 0.3426395	total: 723ms	remaining: 2.55s
221:	learn: 0.

417:	learn: 0.3377944	total: 1.37s	remaining: 1.91s
418:	learn: 0.3377795	total: 1.38s	remaining: 1.91s
419:	learn: 0.3377587	total: 1.38s	remaining: 1.91s
420:	learn: 0.3377451	total: 1.38s	remaining: 1.9s
421:	learn: 0.3377338	total: 1.39s	remaining: 1.9s
422:	learn: 0.3377262	total: 1.39s	remaining: 1.9s
423:	learn: 0.3377114	total: 1.4s	remaining: 1.9s
424:	learn: 0.3376878	total: 1.4s	remaining: 1.89s
425:	learn: 0.3376480	total: 1.4s	remaining: 1.89s
426:	learn: 0.3376385	total: 1.41s	remaining: 1.89s
427:	learn: 0.3376139	total: 1.41s	remaining: 1.88s
428:	learn: 0.3375852	total: 1.41s	remaining: 1.88s
429:	learn: 0.3375639	total: 1.42s	remaining: 1.88s
430:	learn: 0.3375420	total: 1.42s	remaining: 1.87s
431:	learn: 0.3375250	total: 1.42s	remaining: 1.87s
432:	learn: 0.3375113	total: 1.43s	remaining: 1.87s
433:	learn: 0.3375004	total: 1.43s	remaining: 1.86s
434:	learn: 0.3374803	total: 1.43s	remaining: 1.86s
435:	learn: 0.3374605	total: 1.44s	remaining: 1.86s
436:	learn: 0.33743

625:	learn: 0.3340737	total: 2.07s	remaining: 1.24s
626:	learn: 0.3340696	total: 2.07s	remaining: 1.23s
627:	learn: 0.3340537	total: 2.08s	remaining: 1.23s
628:	learn: 0.3340379	total: 2.08s	remaining: 1.23s
629:	learn: 0.3340184	total: 2.08s	remaining: 1.22s
630:	learn: 0.3340044	total: 2.08s	remaining: 1.22s
631:	learn: 0.3339948	total: 2.09s	remaining: 1.22s
632:	learn: 0.3339864	total: 2.09s	remaining: 1.21s
633:	learn: 0.3339837	total: 2.1s	remaining: 1.21s
634:	learn: 0.3339723	total: 2.1s	remaining: 1.21s
635:	learn: 0.3339564	total: 2.1s	remaining: 1.2s
636:	learn: 0.3339445	total: 2.1s	remaining: 1.2s
637:	learn: 0.3339268	total: 2.11s	remaining: 1.2s
638:	learn: 0.3339055	total: 2.11s	remaining: 1.19s
639:	learn: 0.3338917	total: 2.12s	remaining: 1.19s
640:	learn: 0.3338788	total: 2.12s	remaining: 1.19s
641:	learn: 0.3338716	total: 2.12s	remaining: 1.18s
642:	learn: 0.3338533	total: 2.13s	remaining: 1.18s
643:	learn: 0.3338392	total: 2.13s	remaining: 1.18s
644:	learn: 0.33382

833:	learn: 0.3313919	total: 2.76s	remaining: 550ms
834:	learn: 0.3313834	total: 2.77s	remaining: 547ms
835:	learn: 0.3313652	total: 2.77s	remaining: 544ms
836:	learn: 0.3313325	total: 2.77s	remaining: 540ms
837:	learn: 0.3313181	total: 2.78s	remaining: 537ms
838:	learn: 0.3313102	total: 2.78s	remaining: 534ms
839:	learn: 0.3312972	total: 2.79s	remaining: 531ms
840:	learn: 0.3312951	total: 2.79s	remaining: 527ms
841:	learn: 0.3312829	total: 2.79s	remaining: 524ms
842:	learn: 0.3312648	total: 2.79s	remaining: 521ms
843:	learn: 0.3312593	total: 2.8s	remaining: 517ms
844:	learn: 0.3312481	total: 2.8s	remaining: 514ms
845:	learn: 0.3312298	total: 2.81s	remaining: 511ms
846:	learn: 0.3312246	total: 2.81s	remaining: 507ms
847:	learn: 0.3312078	total: 2.81s	remaining: 504ms
848:	learn: 0.3311978	total: 2.81s	remaining: 501ms
849:	learn: 0.3311902	total: 2.82s	remaining: 497ms
850:	learn: 0.3311799	total: 2.82s	remaining: 494ms
851:	learn: 0.3311677	total: 2.82s	remaining: 491ms
852:	learn: 0.

In [20]:
def meta_stacking_model_predict(X) :
    features=pd.DataFrame()
    features[0]=pd.DataFrame(stacking_model_predict(X))
    features[1]=pd.DataFrame(model_for_stacking.predict(X))
    return meta_regressor.predict(features)

In [21]:
y_pred=meta_stacking_model_predict(X_test)
print('MSE = ', mean_squared_error(y_pred, y_test))

MSE =  0.23094288815538702
